In [1]:
# IMPORTS
##########################

%load_ext autoreload
%autoreload 2

import agent
import environment
import doubledqn
import tools
import memory
import simulation
import multiprocessing
import pandas as pd
import os
import json

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from keras import optimizers 

def iter_params(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

Using TensorFlow backend.


In [ ]:
# ONE RUN
#############################################

param = { 
    
    "batch_size" : 30, # 30,50,60,[70]
    "target_update_freq" : 5000,#10000], # 10000,20000,30000,[40000]
    "gamma" : 0.99,# # 0.98,0.99,0.995,[0.999]
    "train_freq" : 1, # 2,3,4,[5]
    "max_size" : 100000,#,70000], # 20000,50000,70000,[100000]
    "max_ep_length" : 2000, # 1000,2000,3000,[4000]
    "policy" : "epsGreedy",#, "epsGreedy"],
    "eps" : 0.1,#, 0.3, 0.1],
    "delta_time" : 10,
    #"optimizer": [optimizers.RMSprop(lr= 0.001), optimizers.Adagrad(), optimizers.Adam()]
}

sumo_RL = simulation.simulator(
                    connection_label = "single_worker",
                    q_network_type = 'simple',
                    target_q_network_type = 'simple',
                    gamma = param["gamma"],
                    target_update_freq = param["target_update_freq"],
                    train_freq = param["train_freq"],
                    num_burn_in = 200,
                    batch_size = param["batch_size"],
                    optimizer = 'adam',
                    loss_func = "mse",
                    max_ep_length = param["max_ep_length"],
                    experiment_id = "Testing_Loading",
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    net_file = "cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "nominal",
                    state_shape = (1,19),
                    num_actions = 2,
                    use_gui = False,
                    delta_time = param["delta_time"],

                   # memory parameters
                    max_size = param["max_size"],

                   # additional parameters

                    policy = param["policy"],
                    eps = param["eps"],
                    num_episodes = 2,
                    monitoring = True,
                    episode_recording = False,
                    hparams = param.keys())

sumo_RL.train()
# agent.load("./logs/First gridsearch/run_144/model_checkpoints/runFirst gridsearch_iter165000.h5")

# agent.ddqn.train(env = agent.env, num_episodes = 100, policy = agent.policy, connection_label = agent.connection_label)

#evaluation_results = agent.evaluate(runs=2, use_gui= False)
#evaluation_results
# data= agent.ddqn.evaluate(env = agent.env, policy = "greedy")

In [2]:
# GRIDSEARCH
###############################################################

experiment_id = "Test_DDQN_Fixed" 


param = { 
    
    "batch_size" : [50,50,50,50,50,50,50], # 30,50,60,[70]
    "target_update_freq" : [10000],# 100000],  #10000], # 10000,20000,30000,[40000]
    "gamma" : [0.99],# # 0.98,0.99,0.995,[0.999]
    "train_freq" : [2], # 2,3,4,[5]
    "max_size" : [1000],#,70000], # 20000,50000,70000,[100000]
    "max_ep_length" : [1000], # 1000,2000,3000,[4000]
    "policy" : ["linDecEpsGreedy"],# "linDecEpsGreedy"],#, "epsGreedy"],
    "eps" : [1],#, 0.3, 0.1],
    "delta_time" : [10],
    "reward" : ["balanced"], # "negative"]
    #"optimizer": [optimizers.RMSprop(lr= 0.001), optimizers.Adagrad(), optimizers.Adam()]
}

param_grid = iter_params(**param)



def worker(input, output):
    """Runs through a chunk of the grid"""

    for position, args in iter(input.get, 'STOP'):
        result = worker_task(position, args)
        output.put(result)


def worker_task(position, args):
    """Tells the worker what to do with grid chunk"""
    # initialise all obje
    
    # print('Run', position + 1, '-- parameters', args)
    
    sumo_RL = simulation.simulator(
                    connection_label = position + 1,
                    q_network_type = 'simple',
                    target_q_network_type = 'simple',
                    gamma = args["gamma"],
                    target_update_freq = args["target_update_freq"],
                    train_freq = args["train_freq"],
                    num_burn_in = 1000,
                    batch_size = args["batch_size"],
                    optimizer = 'adam',
                    loss_func = "mse",
                    max_ep_length = args["max_ep_length"],
                    experiment_id = experiment_id,
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    net_file = "cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "nominal",
                    state_shape = (1,19),
                    num_actions = 2,
                    use_gui = False,
                    delta_time = args["delta_time"],
                    reward = args["reward"],

                   # memory parameters
                    max_size = args["max_size"],

                   # additional parameters

                    policy = args["policy"],
                    eps = args["eps"],
                    num_episodes = 500,
                    monitoring = True,
                    episode_recording = False,
                    hparams = args.keys())
    
    # print("training agent", position + 1)
    sumo_RL.train()
    # print("evaluating agent", position + 1)
    evaluation_results = sumo_RL.evaluate(runs = 5)

    return (position + 1, args, evaluation_results, evaluation_results["average_delay"])


def gridsearch(param_grid):
    """Runs a parallelised gridsearch"""

    number_of_processes = multiprocessing.cpu_count()

    # Set up task list
    tasks = [(idx, val) for idx, val in enumerate(param_grid)]

    # Create queues
    task_queue = multiprocessing.Queue()
    done_queue = multiprocessing.Queue()

    # Submit tasks
    for task in tasks:
        task_queue.put(task)
    # Start worker processes
    for i in range(number_of_processes):
        print('Started process #', i + 1)
        multiprocessing.Process(target = worker,
                                args = (task_queue, done_queue)).start()
        
#     with open(os.path.join("./logs",experiment_id,"GS_results"), "w",newline='') as file:
#             writer = csv.writer(file, dialect = 'excel')
#             writer.writerow(experiment_id)                
      
    # Get and print results
    results = []
    for i in range(len(tasks)):
        results.append(done_queue.get())
        with open(os.path.join("./logs",experiment_id,"GS_results.json"), "a") as file:
            json.dump(results[-1], file , indent=4)
                  
        #print('%s -- [RESULTS]: Run %s -- Parameters %s -- Mean duration %6.0f' % results[-1])
        
    # Tell child processes to stop
    for i in range(number_of_processes):
        task_queue.put('STOP')

    # Now combine the results
    scores = [result[-1] for result in results]
    lowest = min(scores)
    winner = results[scores.index(lowest)]
    return winner, results

multiprocessing.freeze_support()
winner, results = gridsearch(param_grid)

Started process # 1
Started process # 2
Started process # 3
Started process # 4
Started process # 5
Started process # 6
Started process # 7
Started process # 8
...done filling replay memory
Run 3 -- running episode 1 / 500
...done filling replay memory
Run 7 -- running episode 1 / 500
...done filling replay memory
Run 2 -- running episode 1 / 500
...done filling replay memory
Run 4 -- running episode 1 / 500
...done filling replay memory
Run 1 -- running episode 1 / 500
...done filling replay memory
Run 5 -- running episode 1 / 500
...done filling replay memory
Run 6 -- running episode 1 / 500
Run 3 -- running episode 2 / 500
Run 2 -- running episode 2 / 500
Run 7 -- running episode 2 / 500
Run 1 -- running episode 2 / 500
Run 5 -- running episode 2 / 500
Run 6 -- running episode 2 / 500
Run 4 -- running episode 2 / 500
Run 2 -- running episode 3 / 500
Run 3 -- running episode 3 / 500
Run 7 -- running episode 3 / 500
Run 1 -- running episode 3 / 500
Run 4 -- running episode 3 / 500
Run

Run 7 -- running episode 34 / 500
Run 3 -- running episode 34 / 500
Run 5 -- running episode 34 / 500
Run 1 -- running episode 34 / 500
Run 6 -- running episode 34 / 500
Run 4 -- running episode 34 / 500
Run 2 -- running episode 35 / 500
Run 7 -- running episode 35 / 500
Run 3 -- running episode 35 / 500
Run 5 -- running episode 35 / 500
Run 1 -- running episode 35 / 500
Run 6 -- running episode 35 / 500
Run 4 -- running episode 35 / 500
Run 2 -- running episode 36 / 500
Run 7 -- running episode 36 / 500
Run 3 -- running episode 36 / 500
Run 5 -- running episode 36 / 500
Run 1 -- running episode 36 / 500
Run 6 -- running episode 36 / 500
Run 4 -- running episode 36 / 500
Run 2 -- running episode 37 / 500
Run 7 -- running episode 37 / 500
Run 3 -- running episode 37 / 500
Run 5 -- running episode 37 / 500
Run 1 -- running episode 37 / 500
Run 6 -- running episode 37 / 500
Run 4 -- running episode 37 / 500
Run 2 -- running episode 38 / 500
Run 7 -- running episode 38 / 500
Run 3 -- runni

Run 6 -- running episode 68 / 500
Run 1 -- running episode 68 / 500
Run 4 -- running episode 68 / 500
Run 2 -- running episode 69 / 500
Run 3 -- running episode 69 / 500
Run 5 -- running episode 69 / 500
Run 7 -- running episode 69 / 500
Run 6 -- running episode 69 / 500
Run 1 -- running episode 69 / 500
Run 4 -- running episode 69 / 500
Run 3 -- running episode 70 / 500
Run 2 -- running episode 70 / 500
Run 5 -- running episode 70 / 500
Run 7 -- running episode 70 / 500
Run 6 -- running episode 70 / 500
Run 1 -- running episode 70 / 500
Run 4 -- running episode 70 / 500
Run 2 -- running episode 71 / 500
Run 3 -- running episode 71 / 500
Run 5 -- running episode 71 / 500
Run 7 -- running episode 71 / 500
Run 6 -- running episode 71 / 500
Run 1 -- running episode 71 / 500
Run 4 -- running episode 71 / 500
Run 2 -- running episode 72 / 500
Run 3 -- running episode 72 / 500
Run 5 -- running episode 72 / 500
Run 7 -- running episode 72 / 500
Run 6 -- running episode 72 / 500
Run 1 -- runni

Run 2 -- running episode 103 / 500
Run 3 -- running episode 103 / 500
Run 5 -- running episode 103 / 500
Run 1 -- running episode 103 / 500
Run 7 -- running episode 103 / 500
Run 6 -- running episode 103 / 500
Run 4 -- running episode 103 / 500
Run 2 -- running episode 104 / 500
Run 3 -- running episode 104 / 500
Run 5 -- running episode 104 / 500
Run 1 -- running episode 104 / 500
Run 7 -- running episode 104 / 500
Run 6 -- running episode 104 / 500
Run 4 -- running episode 104 / 500
Run 2 -- running episode 105 / 500
Run 3 -- running episode 105 / 500
Run 5 -- running episode 105 / 500
Run 1 -- running episode 105 / 500
Run 7 -- running episode 105 / 500
Run 6 -- running episode 105 / 500
Run 4 -- running episode 105 / 500
Run 2 -- running episode 106 / 500
Run 3 -- running episode 106 / 500
Run 5 -- running episode 106 / 500
Run 1 -- running episode 106 / 500
Run 7 -- running episode 106 / 500
Run 6 -- running episode 106 / 500
Run 4 -- running episode 106 / 500
Run 2 -- running epi

Run 7 -- running episode 136 / 500
Run 4 -- running episode 136 / 500
Run 3 -- running episode 137 / 500
Run 6 -- running episode 136 / 500
Run 1 -- running episode 137 / 500
Run 2 -- running episode 137 / 500
Run 5 -- running episode 137 / 500
Run 7 -- running episode 137 / 500
Run 4 -- running episode 137 / 500
Run 3 -- running episode 138 / 500
Run 6 -- running episode 137 / 500
Run 2 -- running episode 138 / 500
Run 1 -- running episode 138 / 500
Run 5 -- running episode 138 / 500
Run 7 -- running episode 138 / 500
Run 4 -- running episode 138 / 500
Run 3 -- running episode 139 / 500
Run 6 -- running episode 138 / 500
Run 2 -- running episode 139 / 500
Run 1 -- running episode 139 / 500
Run 5 -- running episode 139 / 500
Run 7 -- running episode 139 / 500
Run 4 -- running episode 139 / 500
Run 3 -- running episode 140 / 500
Run 2 -- running episode 140 / 500
Run 6 -- running episode 139 / 500
Run 1 -- running episode 140 / 500
Run 7 -- running episode 140 / 500
Run 5 -- running epi

...

Run 3 -- running episode 382 / 500
Run 6 -- running episode 378 / 500
Run 2 -- running episode 382 / 500
Run 1 -- running episode 377 / 500
Run 7 -- running episode 381 / 500
Run 4 -- running episode 377 / 500
Run 5 -- running episode 324 / 500
Run 3 -- running episode 383 / 500
Run 6 -- running episode 379 / 500
Run 2 -- running episode 383 / 500
Run 1 -- running episode 378 / 500
Run 7 -- running episode 382 / 500
Run 4 -- running episode 378 / 500
Run 5 -- running episode 325 / 500
Run 6 -- running episode 380 / 500
Run 3 -- running episode 384 / 500
Run 2 -- running episode 384 / 500
Run 1 -- running episode 379 / 500
Run 7 -- running episode 383 / 500
Run 4 -- running episode 379 / 500
Run 5 -- running episode 326 / 500
Run 6 -- running episode 381 / 500
Run 3 -- running episode 385 / 500
Run 2 -- running episode 385 / 500
Run 1 -- running episode 380 / 500
Run 7 -- running episode 384 / 500
Run 4 -- running episode 380 / 500
Run 6 -- running episode 382 / 500
Run 5 -- running epi

Run 6 -- running episode 412 / 500
Run 3 -- running episode 416 / 500
Run 2 -- running episode 416 / 500
Run 4 -- running episode 410 / 500
Run 7 -- running episode 415 / 500
Run 1 -- running episode 411 / 500
Run 5 -- running episode 356 / 500
Run 6 -- running episode 413 / 500
Run 3 -- running episode 417 / 500
Run 2 -- running episode 417 / 500
Run 4 -- running episode 411 / 500
Run 7 -- running episode 416 / 500
Run 1 -- running episode 412 / 500
Run 5 -- running episode 357 / 500
Run 6 -- running episode 414 / 500
Run 3 -- running episode 418 / 500
Run 4 -- running episode 412 / 500
Run 2 -- running episode 418 / 500
Run 7 -- running episode 417 / 500
Run 1 -- running episode 413 / 500
Run 5 -- running episode 358 / 500
Run 3 -- running episode 419 / 500
Run 6 -- running episode 415 / 500
Run 4 -- running episode 413 / 500
Run 2 -- running episode 419 / 500
Run 7 -- running episode 418 / 500
Run 1 -- running episode 414 / 500
Run 5 -- running episode 359 / 500
Run 3 -- running epi

Run 6 -- running episode 446 / 500
Run 5 -- running episode 389 / 500
Run 7 -- running episode 447 / 500
Run 3 -- running episode 450 / 500
Run 4 -- running episode 444 / 500
Run 2 -- running episode 450 / 500
Run 1 -- running episode 445 / 500
Run 6 -- running episode 447 / 500
Run 5 -- running episode 390 / 500
Run 7 -- running episode 448 / 500
Run 3 -- running episode 451 / 500
Run 4 -- running episode 445 / 500
Run 2 -- running episode 451 / 500
Run 1 -- running episode 446 / 500
Run 6 -- running episode 448 / 500
Run 5 -- running episode 391 / 500
Run 7 -- running episode 449 / 500
Run 3 -- running episode 452 / 500
Run 4 -- running episode 446 / 500
Run 2 -- running episode 452 / 500
Run 1 -- running episode 447 / 500
Run 6 -- running episode 449 / 500
Run 5 -- running episode 392 / 500
Run 7 -- running episode 450 / 500
Run 3 -- running episode 453 / 500
Run 4 -- running episode 447 / 500
Run 2 -- running episode 453 / 500
Run 1 -- running episode 448 / 500
Run 6 -- running epi

Run 4 -- running episode 477 / 500
Run 1 -- running episode 478 / 500
Run 7 -- running episode 479 / 500
Run 5 -- running episode 423 / 500
Run 3 -- running episode 484 / 500
Run 2 -- running episode 484 / 500
Run 6 -- running episode 481 / 500
Run 4 -- running episode 478 / 500
Run 1 -- running episode 479 / 500
Run 7 -- running episode 480 / 500
Run 5 -- running episode 424 / 500
Run 3 -- running episode 485 / 500
Run 2 -- running episode 485 / 500
Run 6 -- running episode 482 / 500
Run 4 -- running episode 479 / 500
Run 1 -- running episode 480 / 500
Run 7 -- running episode 481 / 500
Run 5 -- running episode 425 / 500
Run 3 -- running episode 486 / 500
Run 2 -- running episode 486 / 500
Run 6 -- running episode 483 / 500
Run 4 -- running episode 480 / 500
Run 1 -- running episode 481 / 500
Run 7 -- running episode 482 / 500
Run 5 -- running episode 426 / 500
Run 3 -- running episode 487 / 500
Run 2 -- running episode 487 / 500
Run 4 -- running episode 481 / 500
Run 6 -- running epi

Evaluate 5 -- running episode 4 / 5
Evaluate 5 -- running episode 5 / 5


In [ ]:
decay_test = []
eps_test = 0.8
for i in range(300000):
    eps_test *= omega_test ** i
    decay_test.append(eps_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(300000),decay_test)

In [ ]:
first_results = pd.DataFrame(results).sort_values(by = 3)